# Learning

In [49]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [67]:
MODEL = 'ResNet50'
batch_size=64
train_data_path = './CargoContainerImages/train/'
num_classes = 2
max_epoches = 20

In [68]:
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode ='categorical')

Found 147 images belonging to 2 classes.


In [69]:
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input
    model = ResNet50(weights = None, classes=num_classes)
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input
    model = VGG16(weights = None, classes = num_classes)
    output_node_name = 'predictions/Softmax'


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/20
3/3 [==============================] - 54s 18s/step - loss: 3.2588 - accuracy: 0.4966
Epoch 2/20
3/3 [==============================] - 33s 11s/step - loss: 3.0985 - accuracy: 0.6054
Epoch 3/20
2/3 [===================>..........] - ETA: 13s - loss: 0.9038 - accuracy: 0.7734

In [16]:
MODEL = 'ResNet50'
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/20
3/3 [==============================] - 34s 11s/step - loss: 1.9850 - accuracy: 0.5034
Epoch 2/20
3/3 [==============================] - 34s 11s/step - loss: 0.6935 - accuracy: 0.5034
Epoch 3/20
3/3 [==============================] - 52s 17s/step - loss: 0.6877 - accuracy: 0.5850
Epoch 4/20
3/3 [==============================] - 54s 18s/step - loss: 0.6852 - accuracy: 0.5850
Epoch 5/20
3/3 [==============================] - 36s 12s/step - loss: 0.6893 - accuracy: 0.5646
Epoch 6/20
3/3 [==============================] - 54s 18s/step - loss: 0.6818 - accuracy: 0.5850
Epoch 7/20
3/3 [==============================] - 53s 18s/step - loss: 0.6811 - accuracy: 0.5850
Epoch 8/20
3/3 [==============================] - 35s 12s/step - loss: 0.6791 - accuracy: 0.5850
Epoch 9/20
3/3 [==============================] - 35s 12s/step - loss: 0.6831 - accuracy: 0.5850
Epoch 10/20
3/3 [==============================] - 52s 17s/step - loss: 0.6812 - accuracy: 0.5850
Epoch 11/20
3/3 [============

In [17]:
mddel.summary()

NameError: name 'mddel' is not defined

# Prediction

In [54]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
#import uff

In [62]:
MODEL = 'ResNet50'

In [63]:
if MODEL == 'ResNet50':
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    output_node_name  = 'predictions/Softmax'
#sess = K.get_session()
#K.set_learning_phase(0)
#graph = tf.get_default_graph()
model = load_model('saved_model/' + MODEL + 'saved.h5')
model.compile(loss = 'categorical_crossenthropy', optimizer = 'adam', metrics= ['accuracy'])
#output_graph_def = tf.graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ['input_1', output_node_name])
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
#uffmodel = uff.from_tensorflow(output_graph_def, [output_node_name], 
#                              output_filename = 'saved_model/' + MODEL + 'uff')

In [64]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import decode_predictions
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import decode_predictions
def predict_all(folder):
    for filename in os.listdir(folder):
        print("Predicting...", filename)
        fullpath = os.path.join(folder, filename)  # 폴더명과 파일명을 결합
        #print("fullpath", fullpath)  
        try:
            # 테스트할 이미지 불러오기
            img = load_img(fullpath, target_size=(224, 224)) 
            # ResNet에 입력하기 전에 이미지 전처리
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # 이미지 분류
            preds = model.predict(x)
            print('horizontal:vertical   ', preds)
            #print('Predicted:', decode_predictions(preds, top=1)[0])

        except Exception as e:
            print(e)

In [65]:
predict_all('./CargoContainerImages/train/1/')

Predicting... BHCU2088072.jpg
horizontal:vertical    [[9.9998808e-01 1.1911531e-05]]
Predicting... CAIU2191783.jpg
horizontal:vertical    [[9.9975926e-01 2.4071608e-04]]
Predicting... CBCU2000317.jpg
horizontal:vertical    [[9.9962294e-01 3.7699656e-04]]
Predicting... CCLU1026613.jpg
horizontal:vertical    [[9.999981e-01 1.925105e-06]]
Predicting... CCLU2058301.jpg
horizontal:vertical    [[0.9924233  0.00757667]]
Predicting... CCLU4755619.jpg
horizontal:vertical    [[0.9987263  0.00127371]]
Predicting... CCLU4856213.jpg
horizontal:vertical    [[0.9973514  0.00264857]]
Predicting... CICU9679964.jpg
horizontal:vertical    [[9.9999833e-01 1.6746067e-06]]
Predicting... CICU9679964_2.jpg
horizontal:vertical    [[0.9988945  0.00110547]]
Predicting... CICU9728204.jpg
horizontal:vertical    [[9.9999952e-01 4.5648238e-07]]
Predicting... CISU2000076.jpg
horizontal:vertical    [[0.99880636 0.0011937 ]]
Predicting... CISU2000203.jpg
horizontal:vertical    [[9.999229e-01 7.710831e-05]]
Predicting..

In [66]:
predict_all('./CargoContainerImages/train/2/')

Predicting... BMOU4699433.jpg
horizontal:vertical    [[0.99591404 0.00408592]]
Predicting... BMOU5782049.jpg
horizontal:vertical    [[9.9961686e-01 3.8315810e-04]]
Predicting... BSIU2575064.jpg
horizontal:vertical    [[9.9976939e-01 2.3056341e-04]]
Predicting... CAIU2728316.jpg
horizontal:vertical    [[9.9993992e-01 6.0104307e-05]]
Predicting... CAIU4051570.jpg
horizontal:vertical    [[0.99722373 0.00277625]]
Predicting... CAIU8121016.jpg
horizontal:vertical    [[9.9974746e-01 2.5248143e-04]]
Predicting... CARU3858347.jpg
horizontal:vertical    [[9.9986672e-01 1.3327455e-04]]
Predicting... CAXU8004650.jpg
horizontal:vertical    [[0.99826115 0.00173889]]
Predicting... CBXU2785862.jpg
horizontal:vertical    [[9.994523e-01 5.476455e-04]]
Predicting... CMAU5349828.jpg
horizontal:vertical    [[0.99899024 0.00100977]]
Predicting... crop002.jpg
horizontal:vertical    [[9.994816e-01 5.183583e-04]]
Predicting... CRXU1525331.jpg
horizontal:vertical    [[9.9938881e-01 6.1111856e-04]]
Predicting..